## Phase 1: Data Ingestion & Clinical Sanitation

Clinical Context & Rationale

Raw healthcare data is designed for billing, not analytics. It often contains artifacts that can severely bias predictive modeling. Before we can calculate risk scores, we must sanitize the dataset to ensure clinical validity.

Key Actions & Justifications:

1. Removing "Weight": This variable has >90% missingness. Imputing it would introduce massive artificial noise, so we remove it entirely to preserve data integrity.

2. Excluding Deceased Patients: Patients who expired during their stay cannot be readmitted. Including them would artificially lower the readmission rate and confuse the model target.

3. Standardizing Identifiers: Numerical IDs (like admission_type_id) are categorical labels, not mathematical values. We treat them as strings/categories to prevent the model from misinterpreting them as quantities.

4. Exporting Clean Data: We save the filtered dataset to a new file (diabetic_data_clean.csv) so subsequent phases can load reliable data directly.

In [4]:
print("\n--- IMPORTING LIBRARIES ---")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import time
import os

# Configuration to display all columns
pd.set_option('display.max_columns', None)


--- IMPORTING LIBRARIES ---


In [5]:
print("\n--- STARTING PHASE 1 ---")

def load_and_sanitize_data(filepath, id_map_path, output_path):
    """
    Loads raw data, handles missing values, removes invalid rows (deceased),
    drops structurally missing columns, AND saves the clean file.
    """
    # 1. Load Data with standardized missing values
    print(f"Loading dataset from: {filepath}")
    df = pd.read_csv(filepath, na_values="?")
    
    # 2. Drop 'weight' due to >90% missingness (Clinical decision)
    if 'weight' in df.columns:
        df = df.drop(columns=['weight'])
        print("- Dropped 'weight' column (>90% missing).")

    # 3. Load ID Mapping to identify discharge codes
    # We define this inside a try-block in case the map file is messy
    try:
        id_map = pd.read_csv(id_map_path, header=None, names=["id", "description"])
        
        # Extract Discharge Disposition block from the mapping file
        # The file is semi-structured, so we find the specific section
        start = id_map[id_map["id"] == "discharge_disposition_id"].index[0] + 1
        end = id_map[id_map["id"] == "admission_source_id"].index[0]
        discharge_map = id_map.iloc[start:end].copy()
        
        # 4. Identify Deceased Codes (Expired + Hospice)
        # We ensure IDs are numeric for matching
        discharge_map["id"] = pd.to_numeric(discharge_map["id"], errors='coerce')
        
        # Search for descriptions containing "Expired" or "Hospice" to be thorough
        # This covers IDs 11, 19, 20, 21 (Expired) and often 13, 14 (Hospice)
        expired_mask = discharge_map["description"].str.contains("Expired|Hospice", case=False, na=False)
        expired_codes = discharge_map[expired_mask]["id"].tolist()
        
        print(f"  > Identified Deceased/Expired Codes to drop: {expired_codes}")
        
        # Filter dataset
        initial_count = len(df)
        df = df[~df['discharge_disposition_id'].isin(expired_codes)]
        print(f"- Removed {initial_count - len(df)} deceased patient records.")
        
    except Exception as e:
        print(f"Warning: Could not filter deceased patients. Error in ID mapping: {e}")

    # 5. Remove exact duplicates (System errors)
    df = df.drop_duplicates()
    
    # 6. Convert categorical IDs to strings (prevent mathematical usage)
    # This prevents the model from thinking ID 2 is "double" ID 1.
    cat_cols = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id']
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype(str)
        
    print(f"Sanitation Complete. Final Shape: {df.shape}")
    
    # 7. EXPORT TO NEW FILE
    df.to_csv(output_path, index=False)
    print(f"✓ Saved clean data to: {output_path}")
    
    return df

# EXECUTION PHASE 1
input_csv = "../data/diabetic_data.csv"
mapping_csv = "../data/IDs_mapping.csv"
output_csv = "../data/diabetic_data_clean.csv"

if os.path.exists(input_csv):
    df_clean = load_and_sanitize_data(
        input_csv, 
        mapping_csv, 
        output_csv
    )
    # Display first few rows to confirm load
    print("\nPreview of Clean Data:")
    print(df_clean.head())
else:
    print(f"Error: '{input_csv}' not found. Please ensure the file is in the notebook directory.")


--- STARTING PHASE 1 ---
Error: '../data/diabetic_data.csv' not found. Please ensure the file is in the notebook directory.


## Phase 2: Clinical Enrichment (ICD-9 Scraper)

Why we need this

The dataset uses ICD-9 codes (e.g., 428, 250.01) to represent diagnoses. These numbers are meaningless to non-clinical stakeholders.

To make our analysis interpretable for VHN administration, we scrape the clinical text descriptions for the most frequent diagnoses. This allows us to say "Heart Failure" instead of "Code 428".

In [ ]:
print("\n--- STARTING PHASE 2 ---")

def fetch_icd9_description(code):
    """
    Scrapes icd9.chrisendres.com to get the text description of a code.
    Includes error handling and delays to be polite to the server.
    """
    base_url = "http://icd9.chrisendres.com/index.php"
    params = {
        'srchtype': 'diseases',
        'srchtext': code,
        'Submit': 'Search',
        'action': 'search'
    }
    
    try:
        response = requests.get(base_url, params=params, headers={"User-Agent": "Chrome/120.0.0.0"})
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            # The site puts descriptions in a div with class 'dlvl'
            div = soup.find('div', class_='dlvl')
            if div:
                # Text usually looks like "038 Septicemia", so we split it
                text = div.text.strip()
                parts = text.split(' ', 1)
                
                # Check match. Note: The website might return "038" even if we searched "38" or "038"
                # So we check if the returned code (parts[0]) matches our input code 
                # OR if our input code matches the returned code without the leading zero.
                returned_code = parts[0]
                if len(parts) > 1 and (returned_code == str(code) or returned_code == str(code).zfill(3)):
                    return parts[1]
    except Exception as e:
        return f"Lookup Failed: {e}"
    
    return "Description Not Found"

def enrich_diagnoses(df, top_n=20):
    """
    Identifies top N codes, scrapes descriptions, and maps them.
    Codes outside the top N are labeled 'Not in Top 20'.
    """
    # Identify top N most frequent diagnosis codes
    top_codes = df['diag_1'].value_counts().head(top_n).index.tolist()
    print(f"Scraping descriptions for top {top_n} diagnoses...")
    
    mapping = {}
    for code in top_codes:
        code_str = str(code)
        
        # Check if code is numeric or special string before scraping
        if code_str != '?' and code_str != 'None':
            # Check length and pad if necessary for the search
            search_term = code_str
            if len(code_str) < 3 and code_str.isdigit():
                search_term = code_str.zfill(3) # "38" -> "038"
             
            desc = fetch_icd9_description(search_term)
             
            # If "Description Not Found" with padded code, try original just in case
            if desc == "Description Not Found" and search_term != code_str:
                desc = fetch_icd9_description(code_str)

            mapping[code_str] = desc
            print(f"Code {code_str} (searched '{search_term}'): {desc}")
            time.sleep(1) # Polite delay
        else:
            mapping[code_str] = "Administrative/Other"
            print(f"Code {code_str}: Administrative/Other")
        
    # Apply mapping:
    # 1. Map the known codes
    # 2. Fill everything else (NaN) with "Not in Top 20"
    df['Primary_Diagnosis_Desc'] = df['diag_1'].map(mapping).fillna("Not in Top 20")
    
    return df

# EXECUTION PHASE 2
# Check for the clean file in the current directory
if os.path.exists("../data/diabetic_data_clean.csv"):
    df_clean = pd.read_csv("../data/diabetic_data_clean.csv")
    
    # Run enrichment with Top 20
    df_clean = enrich_diagnoses(df_clean, top_n=20)
    
    # Show results
    print("\nSample of Enriched Diagnoses:")
    print(df_clean[['diag_1', 'Primary_Diagnosis_Desc']].head(10))
    
    # --- CRITICAL STEP: SAVE THE DATA ---
    df_clean.to_csv("../data/diabetic_data_clean.csv", index=False)
    print("\n✓ Saved enriched data to 'diabetic_data_clean.csv'")
    
else:
    print("Clean data file not found. Run Phase 1 first.")

## Phase 3: Exploratory Data Analysis (EDA)

Objective

We dig into the dataset to uncover the "shape" of the data and identifying the drivers of readmission. This phase moves beyond data cleaning to insight generation.

Why we do this

1. Imbalance Detection: If the target class (<30) is rare, our future models might ignore it. We need to quantify this imbalance.
2. Demographic Disparities: Healthcare outcomes often vary by age, race, and gender. Identifying these disparities is an ethical and clinical necessity.
3. Medication & Operations: Understanding how treatment complexity (Insulin usage, polypharmacy) and hospital operations (Time in hospital, discharge location) impact readmission helps operationalize the findings for VHN.

In [ ]:
print("\n--- STARTING PHASE 3 ---")

# Set visual style for professional charts
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Load the clean data
# Note: Using the specific path requested
file_path = "../data/diabetic_data_clean.csv"

if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    print(f"Data Loaded. Shape: {df.shape}")
else:
    print(f"Error: Clean data file not found at {file_path}. Please run Phase 1 & 2 or check the path.")

# --- HELPER FUNCTIONS ---

def plot_readmission_landscape(df):
    """
    1. The Readmission Landscape: Analyzes the target variable distribution.
    Why: To identify class imbalance which affects model choice and metric selection.
    """
    plt.figure(figsize=(8, 5))
    
    # Calculate percentages
    counts = df['readmitted'].value_counts(normalize=True) * 100
    
    ax = sns.countplot(x='readmitted', data=df, order=['NO', '>30', '<30'], palette='viridis')
    plt.title('Distribution of Readmission Classes', fontsize=14)
    plt.ylabel('Patient Count')
    plt.xlabel('Readmission Status')
    
    # Add percentage labels
    for p in ax.patches:
        ax.annotate(f'{p.get_height() / len(df) * 100:.1f}%', 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha = 'center', va = 'center', xytext = (0, 5), textcoords = 'offset points')
    
    plt.show()
    print(f"Insight: The target class (<30 days) represents {counts.get('<30', 0):.1f}% of the data.")
    print("This indicates a moderate class imbalance that may require stratification or resampling later.")
    
    print("\nDetailed Readmission Rates:")
    print(df['readmitted'].value_counts(normalize=True) * 100)

def analyze_demographics(df):
    """
    2. Demographic Profiling: Age, Race, and Gender analysis.
    Why: To detect if specific subgroups are disproportionately affecting readmission rates.
    """
    # Age Distribution
    plt.figure(figsize=(12, 5))
    sns.countplot(x='age', data=df, palette='coolwarm')
    plt.title('Age Distribution of Patient Cohort')
    plt.show()

    # Race & Gender Stratification
    # We create a pivot table to see the rate of <30 readmissions
    df['target'] = (df['readmitted'] == '<30').astype(int)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='race', y='target', hue='gender', data=df, ci=None, palette='muted')
    plt.title('30-Day Readmission Rate by Race and Gender')
    plt.ylabel('Readmission Rate (<30)')
    plt.legend(title='Gender', loc='upper right')
    plt.show()
    
    # Textual Insight
    rates = df.groupby(['race', 'gender'])['target'].mean() * 100
    print("Readmission Rates (<30) Summary:")
    print(rates)

def analyze_medication_efficacy(df):
    """
    3. Medication Efficacy: Insulin vs Oral vs No Meds.
    Why: Insulin usage often signals advanced disease progression (severity).
    """
    # Define Logic: 
    # If Insulin != No -> 'Insulin' (High Severity)
    # Else If any other med != No -> 'Oral Only'
    # Else -> 'No Meds'
    
    # List of 22 oral meds (excluding insulin)
    # These represent the full list of diabetes medications recorded in the dataset
    oral_meds = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 
                 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 
                 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 
                 'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 
                 'glipizide-metformin', 'glimepiride-pioglitazone', 
                 'metformin-rosiglitazone', 'metformin-pioglitazone']
    
    def classify_meds(row):
        # Priority 1: Insulin usage indicates higher severity/complexity
        if row['insulin'] != 'No':
            return 'Insulin (Any)'
        
        # Priority 2: Check if any oral med is taken
        # We iterate through all 22 oral medication columns. 
        # If any of them are NOT 'No', the patient is on oral meds.
        for med in oral_meds:
            if med in row and row[med] != 'No':
                return 'Oral Only'
        
        # Priority 3: Neither insulin nor oral meds
        return 'No Medication'

    df['med_status'] = df.apply(classify_meds, axis=1)

    # Visualization 1: Insulin vs Oral
    plt.figure(figsize=(8, 5))
    sns.barplot(x='med_status', y='target', data=df, ci=None, palette='Set2')
    plt.title('Readmission Risk by Medication Type')
    plt.ylabel('Readmission Rate (<30)')
    plt.show()

    # Visualization 2: Change in Meds
    plt.figure(figsize=(6, 5))
    sns.barplot(x='change', y='target', data=df, ci=None, palette='pastel')
    plt.title('Readmission Risk: Medication Change vs No Change')
    plt.ylabel('Readmission Rate (<30)')
    plt.show()

def analyze_operational_metrics(df):
    """
    4. Operational Metrics: Time in hospital, Lab procedures, Discharge location.
    Why: To find operational bottlenecks or indicators of complex stays.
    """
    # Scatter: Time vs Labs
    plt.figure(figsize=(8, 6))
    # Using hexbin for clearer density visualization with large data
    plt.hexbin(df['time_in_hospital'], df['num_lab_procedures'], gridsize=20, cmap='Blues')
    plt.colorbar(label='Count')
    plt.title('Time in Hospital vs Number of Lab Procedures')
    plt.xlabel('Days in Hospital')
    plt.ylabel('Num Lab Procedures')
    plt.show()

    # Correlation Heatmap
    cols = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_diagnoses']
    corr = df[cols].corr()
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlation Heatmap of Numerical Features')
    plt.show()

    # Box Plot: Time in Hospital by Readmission
    plt.figure(figsize=(8, 5))
    sns.boxplot(x='readmitted', y='time_in_hospital', data=df, order=['NO', '>30', '<30'], palette='Set3')
    plt.title('Length of Stay Distribution by Readmission Status')
    plt.show()

    # Discharge Disposition: Home (1) vs SNF (3)
    # Filter for just these two common codes for clear comparison
    subset = df[df['discharge_disposition_id'].isin(['1', '3'])].copy()
    subset['location'] = subset['discharge_disposition_id'].map({'1': 'Home', '3': 'Skilled Nursing Facility'})
    
    plt.figure(figsize=(8, 5))
    sns.barplot(x='location', y='target', data=subset, ci=None, palette='autumn')
    plt.title('Readmission Rate: Home vs Skilled Nursing Facility (SNF)')
    plt.ylabel('Readmission Rate (<30)')
    plt.show()

# --- EXECUTION ---
if 'df' in locals():
    print("--- 1. Readmission Landscape ---")
    plot_readmission_landscape(df)
    
    print("\n--- 2. Demographic Profiling ---")
    analyze_demographics(df)
    
    print("\n--- 3. Medication Efficacy ---")
    analyze_medication_efficacy(df)
    
    print("\n--- 4. Operational Metrics ---")
    analyze_operational_metrics(df)

## Phase 4: Feature Engineering – Vitality Complexity Index (VCI)

Operational GoalNursing leadership requires a simple, rule-based score to identify high-risk patients on the floor. Machine learning models are often "black boxes"; the VCI is designed to be transparent.

Scoring Logic (The "Why")

We calculate points based on clinical proxies for instability:
1. L (Length of Stay): Longer stays imply complications ($>14$ days = 7 points).
2. A (Acuity): Emergency admissions indicate an unstable start state (Emergency = 3 points).
3. C (Comorbidity): More diagnoses suggest systemic failure ($>8$ diagnoses = 5 points).
4. E (Emergency Visits): Frequent ED usage shows poor outpatient control ($>4$ visits = 5 points).

##### Total VCI = L + A + C + E

In [ ]:
print("\n--- STARTING PHASE 4 ---")

# --- VCI Helper Functions ---

def calculate_L_points(days):
    """L: Length of Stay points"""
    if days < 1: return 0
    if 1 <= days <= 4: return 1
    if 5 <= days <= 13: return 4
    return 7 # >= 14 days

def calculate_A_points(admission_id):
    """A: Acuity points (Emergency=1, Trauma=7 get points)"""
    # Note: admission_id is string because we sanitized it in Phase 1
    if str(admission_id) in ['1', '7']:
        return 3
    return 0

def calculate_C_points(num_diagnoses):
    """C: Comorbidity Burden points"""
    if num_diagnoses < 4: return 0
    if 4 <= num_diagnoses <= 7: return 3
    return 5 # >= 8 diagnoses

def calculate_E_points(num_emergency):
    """E: Emergency Visit History points"""
    if num_emergency == 0: return 0
    if 1 <= num_emergency <= 4: return 3
    return 5 # > 4 visits

def assign_risk_category(score):
    """Categorizes VCI Score into Clinical Risk Groups"""
    if score < 7: return "Low Risk"
    if 7 <= score <= 10: return "Medium Risk"
    return "High Risk"

# --- Main Feature Engineering Function ---

def apply_vci_scoring(df):
    """
    Applies the VCI framework to the dataframe.
    """
    print("Calculating Vitality Complexity Index (VCI)...")
    
    # 1. Calculate Component Points
    df['L_points'] = df['time_in_hospital'].apply(calculate_L_points)
    df['A_points'] = df['admission_type_id'].apply(calculate_A_points)
    df['C_points'] = df['number_diagnoses'].apply(calculate_C_points)
    df['E_points'] = df['number_emergency'].apply(calculate_E_points)
    
    # 2. Sum for Total Score
    df['VCI_Score'] = df['L_points'] + df['A_points'] + df['C_points'] + df['E_points']
    
    # 3. Stratify Risk
    df['VCI_Risk_Category'] = df['VCI_Score'].apply(assign_risk_category)
    
    return df

# EXECUTION PHASE 4
if 'df_clean' in locals():
    df_final = apply_vci_scoring(df_clean)

    # Validate Distribution
    print("\nRisk Category Distribution:")
    print(df_final['VCI_Risk_Category'].value_counts(normalize=True))
else:
    print("Dataframe not loaded.")

Validation: Does VCI actually predict readmission?

To confirm the VCI is valid, we check the actual readmission rates for each group. We expect a monotonic increase (Low < Medium < High).

In [ ]:
if 'df_final' in locals():
    # Calculate Readmission Rates by Category
    # Note: Target class is '<30' (Readmitted within 30 days)
    # We use normalize=True to get percentages
    validation = df_final.groupby('VCI_Risk_Category')['readmitted'].value_counts(normalize=True).unstack()
    
    if '<30' in validation.columns:
        validation['Rate <30 Days (%)'] = validation['<30'] * 100

        # Visualize
        plt.figure(figsize=(8, 5))
        colors = ['green', 'orange', 'red']
        # Reorder index for logical flow
        order = ['Low Risk', 'Medium Risk', 'High Risk']
        
        # Ensure only existing categories are plotted
        existing_order = [o for o in order if o in validation.index]
        
        validation.loc[existing_order, 'Rate <30 Days (%)'].plot(kind='bar', color=colors, edgecolor='black')

        plt.title('Validation: 30-Day Readmission Rate by VCI Risk Group')
        plt.ylabel('Readmission Rate (%)')
        plt.xlabel('Risk Tier')
        plt.grid(axis='y', alpha=0.3)
        plt.show()

        print("The chart confirms monotonic increase: High Risk patients are significantly more likely to be readmitted.")
    else:
        print("Target class '<30' not found in data.")
else:
    print("Dataframe not loaded.")

In [ ]:
## Phase 3.5: Quantification of Key Risk Signals
# PURPOSE: Extract numeric metrics for the Strategic Insight Report (NO VISUALS)

# =========================
# 1. Overall Readmission Rate
# =========================
overall_rate = (df["readmitted"] == "<30").mean() * 100
print(f"Overall 30-day readmission rate: {overall_rate:.2f}%")

# =========================
# 2. Medication Severity Signal
# =========================
med_readmission = (
    df.groupby("med_status")["readmitted"]
      .apply(lambda x: (x == "<30").mean() * 100)
      .round(2)
)

print("\n30-day readmission rate by medication status:")
print(med_readmission)

if "Insulin" in med_readmission and "No Medication" in med_readmission:
    med_risk_ratio = med_readmission["Insulin"] / med_readmission["No Medication"]
    print(f"\nRelative risk (Insulin vs No Medication): {med_risk_ratio:.2f}x")

# =========================
# 3. Admission Acuity Signal
# =========================
df["emergency_admission"] = df["admission_type_id"].isin([1, 7])

acuity_rates = (
    df.groupby("emergency_admission")["readmitted"]
      .apply(lambda x: (x == "<30").mean() * 100)
      .round(2)
)

print("\n30-day readmission rate by admission acuity:")
print(acuity_rates.rename({True: "Emergency", False: "Non-Emergency"}))

# =========================
# 4. Discharge Destination Risk
# =========================
subset = df[df["discharge_disposition_id"].isin([1, 3])].copy()

subset["discharge_location"] = subset["discharge_disposition_id"].map({
    1: "Home",
    3: "Skilled Nursing Facility"
})

discharge_rates = (
    subset.groupby("discharge_location")["readmitted"]
          .apply(lambda x: (x == "<30").mean() * 100)
          .round(2)
)

print("\n30-day readmission rate by discharge location:")
print(discharge_rates)

# =========================
# 5. Recompute VCI (Required for Report)
# =========================

# Ensure numeric
df["admission_type_id"] = pd.to_numeric(df["admission_type_id"], errors="coerce")
df["number_emergency"] = pd.to_numeric(df["number_emergency"], errors="coerce")

def calculate_L(days):
    if days < 1:
        return 0
    elif 1 <= days <= 4:
        return 1
    elif 5 <= days <= 13:
        return 4
    else:
        return 7

def calculate_A(adm):
    return 3 if adm in [1, 7] else 0

def calculate_C(diag):
    if diag < 4:
        return 0
    elif diag <= 7:
        return 3
    else:
        return 5

def calculate_E(em):
    if em == 0:
        return 0
    elif em <= 4:
        return 3
    else:
        return 5

df["VCI_Score"] = (
    df["time_in_hospital"].apply(calculate_L)
    + df["admission_type_id"].apply(calculate_A)
    + df["number_diagnoses"].apply(calculate_C)
    + df["number_emergency"].apply(calculate_E)
)

def assign_risk(score):
    if score < 7:
        return "Low Risk"
    elif score <= 10:
        return "Medium Risk"
    else:
        return "High Risk"

df["VCI_Risk_Category"] = df["VCI_Score"].apply(assign_risk)

# =========================
# 6. VCI Validation Metrics
# =========================
vci_population = (
    df["VCI_Risk_Category"]
      .value_counts(normalize=True)
      .mul(100)
      .round(2)
)

vci_readmission = (
    df.groupby("VCI_Risk_Category")["readmitted"]
      .apply(lambda x: (x == "<30").mean() * 100)
      .round(2)
)

vci_summary = pd.DataFrame({
    "Population %": vci_population,
    "30-Day Readmission %": vci_readmission
}).loc[["Low Risk", "Medium Risk", "High Risk"]]

print("\nVCI Risk Stratification Summary:")
print(vci_summary)

vci_risk_ratio = (
    vci_summary.loc["High Risk", "30-Day Readmission %"] /
    vci_summary.loc["Low Risk", "30-Day Readmission %"]
)

print(f"\nRelative risk (High vs Low VCI): {vci_risk_ratio:.2f}x")
